In [1]:
import pandas as pd
import numpy as np

In [66]:
df = pd.read_csv("./csv_folder/table2.csv")

In [87]:
df.head()

,icustay_id,charttime,heartrate,sysbp,diasbp,meanbp,resprate,tempc,spo2,glucose,intime,chartoffset
0,200003,2199-08-02 19:15:00,132.0,NaN,NaN,NaN,25.0,39.555554,97.0,NaN,2199-08-02 19:50:04,-35.066667
1,200003,2199-08-02 20:00:00,119.0,NaN,NaN,NaN,35.0,NaN,97.0,NaN,2199-08-02 19:50:04,9.933333
2,200003,2199-08-02 21:00:00,122.0,NaN,NaN,NaN,34.0,38.999998,97.0,NaN,2199-08-02 19:50:04,69.933333
3,200003,2199-08-02 21:25:00,115.0,NaN,NaN,NaN,33.0,NaN,96.0,NaN,2199-08-02 19:50:04,94.933333
4,200003,2199-08-02 21:30:00,116.0,NaN,NaN,NaN,34.0,NaN,95.0,NaN,2199-08-02 19:50:04,99.933333


In [29]:
#df[(df["patientunitstayid"]==141194)&(df['chartoffset']>=0)&(df['chartoffset']<=1440)]

In [68]:
df_group = df.groupby("icustay_id")

In [8]:
def calc_percent(df, timerange=[0, 1440], col='sysbp', valuerange=[70, 80]):
    start = timerange[0]
    end = timerange[1]
    temp = df[(df['chartoffset']>=start) & (df['chartoffset']<=end)]
    non_na = temp[temp[col].notnull()]
    num_nonna = non_na.shape[0]
    if num_nonna==0:
        return np.nan
    value_min, value_max = valuerange[0], valuerange[1]
    range_num = non_na[(non_na[col]>=value_min) & (non_na[col]<=value_max)].shape[0]
    return range_num/num_nonna

In [88]:
def calc_percent_below(df, timerange=[0, 1440], col='sysbp', valuerange=[70, 80]):
    start = timerange[0]
    end = timerange[1]
    temp = df[(df['chartoffset']>=start) & (df['chartoffset']<=end)]
    non_na = temp[temp[col].notnull()]
    num_nonna = non_na.shape[0]
    if ((num_nonna==0) ):
        return np.nan
    value_min= valuerange[0]
    below_num = non_na[(non_na[col]<value_min)].shape[0]
    return below_num/num_nonna

In [89]:
def calc_percent_above(df, timerange=[0, 1440], col='sysbp', valuerange=[70, 80]):
    start = timerange[0]
    end = timerange[1]
    temp = df[(df['chartoffset']>=start) & (df['chartoffset']<=end)]
    non_na = temp[temp[col].notnull()]
    num_nonna = non_na.shape[0]
    if ((num_nonna==0)  ):
        return np.nan
    value_max= valuerange[1]
    above_num = non_na[(non_na[col]>value_max)].shape[0]
    return above_num/num_nonna

In [90]:
below = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 1440], col='meanbp', valuerange=[70,80]))

In [91]:
above = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 1440], col='meanbp', valuerange=[70,80]))

In [93]:
below.head(1)

,icustay_id,propBelow_map_24
0,200003,0.058824


In [92]:
below = pd.DataFrame(below).reset_index()
above = pd.DataFrame(above).reset_index()
below.columns = ["icustay_id", "propBelow_map_24"]
above.columns = ["icustay_id", "propAbove_map_24"]
result_df = below.copy()
result_df['propAbove_map_24'] = above['propAbove_map_24']
result_df['prop_map_24'] = 1-result_df['propAbove_map_24'] -result_df['propBelow_map_24']

In [94]:
result_df.head()

,icustay_id,propBelow_map_24,propAbove_map_24,prop_map_24
0,200003,0.058824,0.470588,0.470588
1,200007,NaN,NaN,NaN
2,200009,0.548387,0.258065,0.193548
3,200012,NaN,NaN,NaN
4,200014,0.250000,0.571429,0.178571


In [95]:
sbp_24_below = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 1440], col='sysbp', valuerange=[110,120]))
sbp_24_below = pd.DataFrame(sbp_24_below).reset_index()
sbp_24_below.columns = ["icustay_id", "propBelow_asp_24"]
sbp_24_above = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 1440], col='sysbp', valuerange=[110,120]))
sbp_24_above = pd.DataFrame(sbp_24_above).reset_index()
sbp_24_above.columns = ["icustay_id", "propAbove_asp_24"]
result_df['propBelow_asp_24'] = sbp_24_below['propBelow_asp_24']
result_df['propAbove_asp_24'] = sbp_24_above['propAbove_asp_24']
result_df['prop_asp_24'] = 1-result_df['propAbove_asp_24'] -result_df['propBelow_asp_24']

In [77]:
result_df.head()

,icustay_id,propBelow_map_24,propAbove_map_24,prop_map_24,propBelow_asp_24,propAbove_asp_24,prop_asp_24
0,200003,0.058824,0.882353,0.058824,0.687500,0.312500,0.000000e+00
1,200007,NaN,NaN,NaN,NaN,NaN,NaN
2,200009,0.645161,0.354839,0.000000,0.774194,0.225806,0.000000e+00
3,200012,NaN,NaN,NaN,NaN,NaN,NaN
4,200014,0.250000,0.678571,0.071429,0.321429,0.678571,-5.551115e-17


In [96]:
adp_24_below = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 1440], col='diasbp', valuerange=[50,60]))
adp_24_below = pd.DataFrame(adp_24_below).reset_index()
adp_24_below.columns = ["icustay_id", "propBelow_adp_24"]
adp_24_above = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 1440], col='diasbp', valuerange=[50,60]))
adp_24_above = pd.DataFrame(adp_24_above).reset_index()
adp_24_above.columns = ["icustay_id", "propAbove_adp_24"]
result_df['propBelow_adp_24'] = adp_24_below['propBelow_adp_24']
result_df['propAbove_adp_24'] = adp_24_above['propAbove_adp_24']
result_df['prop_adp_24'] = 1-result_df['propAbove_adp_24'] -result_df['propBelow_adp_24']

In [97]:
result_df.head()

,icustay_id,propBelow_map_24,propAbove_map_24,prop_map_24,propBelow_asp_24,propAbove_asp_24,prop_asp_24,propBelow_adp_24,propAbove_adp_24,prop_adp_24
0,200003,0.058824,0.470588,0.470588,0.437500,0.187500,0.375000,0.062500,0.750000,0.187500
1,200007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200009,0.548387,0.258065,0.193548,0.709677,0.161290,0.129032,0.032258,0.354839,0.612903
3,200012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200014,0.250000,0.571429,0.178571,0.321429,0.607143,0.071429,0.071429,0.535714,0.392857


In [98]:
result_df.to_csv("./csv_folder/mimic_prop_24h.csv")

# 48hours

In [101]:
below_48 = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 2880], col='meanbp', valuerange=[70,80]))
above_48 = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 2880], col='meanbp', valuerange=[70,80]))
below_48 = pd.DataFrame(below_48).reset_index()
above_48 = pd.DataFrame(above_48).reset_index()
below_48.columns = ["icustay_id", "propBelow_map_48"]
above_48.columns = ["icustay_id", "propAbove_map_48"]
result_df_48 = below_48.copy()
result_df_48['propAbove_map_48'] = above_48['propAbove_map_48']
result_df_48['prop_map_48'] = 1-result_df_48['propAbove_map_48'] -result_df_48['propBelow_map_48']

In [102]:
sbp_48_below = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 2880], col='sysbp', valuerange=[110,120]))
sbp_48_below = pd.DataFrame(sbp_48_below).reset_index()
sbp_48_below.columns = ["icustay_id", "propBelow_asp_48"]
sbp_48_above = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 2880], col='sysbp', valuerange=[110,120]))
sbp_48_above = pd.DataFrame(sbp_48_above).reset_index()
sbp_48_above.columns = ["icustay_id", "propAbove_asp_48"]
result_df_48['propBelow_asp_48'] = sbp_48_below['propBelow_asp_48']
result_df_48['propAbove_asp_48'] = sbp_48_above['propAbove_asp_48']
result_df_48['prop_asp_48'] = 1-result_df_48['propAbove_asp_48'] -result_df_48['propBelow_asp_48']

In [103]:
adp_48_below = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 2880], col='diasbp', valuerange=[50,60]))
adp_48_below = pd.DataFrame(adp_48_below).reset_index()
adp_48_below.columns = ["icustay_id", "propBelow_adp_48"]
adp_48_above = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 2880], col='diasbp', valuerange=[50,60]))
adp_48_above = pd.DataFrame(adp_48_above).reset_index()
adp_48_above.columns = ["icustay_id", "propAbove_adp_48"]
result_df_48['propBelow_adp_48'] = adp_48_below['propBelow_adp_48']
result_df_48['propAbove_adp_48'] = adp_48_above['propAbove_adp_48']
result_df_48['prop_adp_48'] = 1-result_df_48['propAbove_adp_48'] -result_df_48['propBelow_adp_48']

In [105]:
result_df_48.head()

,icustay_id,propBelow_map_48,propAbove_map_48,prop_map_48,propBelow_asp_48,propAbove_asp_48,prop_asp_48,propBelow_adp_48,propAbove_adp_48,prop_adp_48
0,200003,0.176471,0.254902,0.568627,0.680000,0.080000,0.240000,0.020000,0.500000,0.480000
1,200007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200009,0.309091,0.236364,0.454545,0.625000,0.125000,0.250000,0.017857,0.589286,0.392857
3,200012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200014,0.162791,0.720930,0.116279,0.209302,0.744186,0.046512,0.046512,0.697674,0.255814


In [106]:
result_df_48.to_csv("./csv_folder/mimic_prop_48h.csv")

# 72 hours

In [107]:
below_72 = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 4320], col='meanbp', valuerange=[70,80]))
above_72 = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 4320], col='meanbp', valuerange=[70,80]))
below_72 = pd.DataFrame(below_72).reset_index()
above_72 = pd.DataFrame(above_72).reset_index()
below_72.columns = ["icustay_id", "propBelow_map_72"]
above_72.columns = ["icustay_id", "propAbove_map_72"]
result_df_72 = below_72.copy()
result_df_72['propAbove_map_72'] = above_72['propAbove_map_72']
result_df_72['prop_map_72'] = 1-result_df_72['propAbove_map_72'] -result_df_72['propBelow_map_72']

In [108]:
sbp_72_below = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 4320], col='sysbp', valuerange=[110,120]))
sbp_72_below = pd.DataFrame(sbp_72_below).reset_index()
sbp_72_below.columns = ["icustay_id", "propBelow_asp_72"]
sbp_72_above = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 4320], col='sysbp', valuerange=[110,120]))
sbp_72_above = pd.DataFrame(sbp_72_above).reset_index()
sbp_72_above.columns = ["icustay_id", "propAbove_asp_72"]
result_df_72['propBelow_asp_72'] = sbp_72_below['propBelow_asp_72']
result_df_72['propAbove_asp_72'] = sbp_72_above['propAbove_asp_72']
result_df_72['prop_asp_72'] = 1-result_df_72['propAbove_asp_72'] -result_df_72['propBelow_asp_72']

In [109]:
adp_72_below = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 4320], col='diasbp', valuerange=[50,60]))
adp_72_below = pd.DataFrame(adp_72_below).reset_index()
adp_72_below.columns = ["icustay_id", "propBelow_adp_72"]
adp_72_above = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 4320], col='diasbp', valuerange=[50,60]))
adp_72_above = pd.DataFrame(adp_72_above).reset_index()
adp_72_above.columns = ["icustay_id", "propAbove_adp_72"]
result_df_72['propBelow_adp_72'] = adp_72_below['propBelow_adp_72']
result_df_72['propAbove_adp_72'] = adp_72_above['propAbove_adp_72']
result_df_72['prop_adp_72'] = 1-result_df_72['propAbove_adp_72'] -result_df_72['propBelow_adp_72']

In [110]:
result_df_72.head()

,icustay_id,propBelow_map_72,propAbove_map_72,prop_map_72,propBelow_asp_72,propAbove_asp_72,prop_asp_72,propBelow_adp_72,propAbove_adp_72,prop_adp_72
0,200003,0.116883,0.324675,0.558442,0.578947,0.078947,0.342105,0.013158,0.592105,0.394737
1,200007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200009,0.309091,0.236364,0.454545,0.625000,0.125000,0.250000,0.017857,0.589286,0.392857
3,200012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200014,0.162791,0.720930,0.116279,0.209302,0.744186,0.046512,0.046512,0.697674,0.255814


In [111]:
result_df_72.to_csv("./csv_folder/mimic_prop_72h.csv")

# below not used

In [68]:
mbp_48_inrange = df_group.apply(lambda x: calc_percent(x, timerange=[0, 2880], col='ibp_mean', valuerange=[78.27889, 98.53015]))
mbp_48_inrange = pd.DataFrame(mbp_48_inrange).reset_index()
mbp_48_inrange.columns = ["icustay_id", "mbp_48_inrange"]
result_df['mbp_48_inrange'] = mbp_48_inrange['mbp_48_inrange']

sbp_48_inrange = df_group.apply(lambda x: calc_percent(x, timerange=[0, 2880], col='ibp_systolic', valuerange=[122.4724, 154.3116]))
sbp_48_inrange = pd.DataFrame(sbp_48_inrange).reset_index()
sbp_48_inrange.columns = ["icustay_id", "sbp_48_inrange"]
result_df['sbp_48_inrange'] = sbp_48_inrange['sbp_48_inrange']

dbp_48_inrange = df_group.apply(lambda x: calc_percent(x, timerange=[0, 2880], col='ibp_diastolic', valuerange=[60.39196, 79.55276]))
dbp_48_inrange = pd.DataFrame(dbp_48_inrange).reset_index()
dbp_48_inrange.columns = ["icustay_id", "dbp_48_inrange"]
result_df['dbp_48_inrange'] = dbp_48_inrange['dbp_48_inrange']

mbp_72_inrange = df_group.apply(lambda x: calc_percent(x, timerange=[0, 4320], col='ibp_mean', valuerange=[79.39698, 93.25628]))
mbp_72_inrange = pd.DataFrame(mbp_72_inrange).reset_index()
mbp_72_inrange.columns = ["icustay_id", "mbp_72_inrange"]
result_df['mbp_72_inrange'] = mbp_72_inrange['mbp_72_inrange']

sbp_72_inrange = df_group.apply(lambda x: calc_percent(x, timerange=[0, 4320], col='ibp_systolic', valuerange=[123.4673, 153.3166]))
sbp_72_inrange = pd.DataFrame(sbp_72_inrange).reset_index()
sbp_72_inrange.columns = ["icustay_id", "sbp_72_inrange"]
result_df['sbp_72_inrange'] = sbp_72_inrange['sbp_72_inrange']

dbp_72_inrange = df_group.apply(lambda x: calc_percent(x, timerange=[0, 4320], col='ibp_diastolic', valuerange=[61.01005, 82.64322]))
dbp_72_inrange = pd.DataFrame(dbp_72_inrange).reset_index()
dbp_72_inrange.columns = ["icustay_id", "dbp_72_inrange"]
result_df['dbp_72_inrange'] = dbp_72_inrange['dbp_72_inrange']

In [69]:
result_df.head()

,patientunitstayid,mbp_24_inrange,sbp_24_inrange,dbp_24_inrange,mbp_48_inrange,sbp_48_inrange,dbp_48_inrange,mbp_72_inrange,sbp_72_inrange,dbp_72_inrange
0,141168,0.700000,NaN,NaN,0.533333,0.200000,0.200000,0.466667,0.200000,0.200000
1,141194,0.000000,0.041667,0.000000,0.153846,0.098039,0.078431,0.153846,0.098039,0.078431
2,141203,0.417582,NaN,NaN,0.316832,NaN,NaN,0.247525,NaN,NaN
3,141229,1.000000,NaN,NaN,0.500000,NaN,NaN,0.500000,NaN,NaN
4,141233,0.364486,0.457944,0.037383,0.131068,0.150485,0.009709,0.122881,0.114407,0.084746


In [73]:
result_df[result_df['mbp_24_inrange'].notnull()].shape

(20884, 10)

In [12]:
df1 = pd.read_csv("./csv_folder/mimic_clean_19Aug.csv", index_col=0)

In [13]:
df1.head()

,subject_id,hadm_id,icustay_id,gender,admittime,dischtime,los_hospital,age,ethnicity,admission_type,...,exclude_tempc,ibp_systolic_24h,ibp_systolic_48h,ibp_systolic_72h,ibp_diastolic_24h,ibp_diastolic_48h,ibp_diastolic_72h,ibp_mean_24h,ibp_mean_48h,ibp_mean_72h
0,55853,100010,271147,F,2109-12-10 07:15:00,2109-12-14 16:45:00,4.3958,54.5208,WHITE,ELECTIVE,...,False,100.5,99.0,99.0,47.0,50.0,50.0,66.5,68.0,68.0
1,87977,100011,214619,M,2177-08-29 04:51:00,2177-09-12 14:24:00,14.3979,21.5041,HISPANIC OR LATINO,EMERGENCY,...,False,127.0,147.0,148.0,67.0,72.0,72.0,84.0,91.0,92.0
2,14121,100058,297937,F,2139-09-08 21:38:00,2139-09-26 13:55:00,17.6785,56.8114,WHITE,EMERGENCY,...,False,108.0,108.0,108.0,59.0,58.0,57.0,79.0,78.0,77.0
3,20683,100186,203786,M,2168-01-07 19:33:00,2168-01-12 14:00:00,4.7688,19.8138,WHITE,EMERGENCY,...,False,122.0,118.5,118.5,59.5,59.0,59.0,77.0,77.0,77.0
4,81865,100215,248622,M,2193-05-03 20:47:00,2193-05-15 18:31:00,11.9056,48.0116,WHITE,EMERGENCY,...,False,113.5,113.0,113.0,69.0,66.0,66.0,82.0,80.0,81.0


In [15]:
df3 = pd.merge(df1, result_df, how='left', on=['icustay_id'])

In [16]:
df3.head()

,subject_id,hadm_id,icustay_id,gender,admittime,dischtime,los_hospital,age,ethnicity,admission_type,...,ibp_systolic_72h,ibp_diastolic_24h,ibp_diastolic_48h,ibp_diastolic_72h,ibp_mean_24h,ibp_mean_48h,ibp_mean_72h,mbp_24_inrange,sbp_24_inrange,dbp_24_inrange
0,55853,100010,271147,F,2109-12-10 07:15:00,2109-12-14 16:45:00,4.3958,54.5208,WHITE,ELECTIVE,...,99.0,47.0,50.0,50.0,66.5,68.0,68.0,0.166667,0.208333,0.041667
1,87977,100011,214619,M,2177-08-29 04:51:00,2177-09-12 14:24:00,14.3979,21.5041,HISPANIC OR LATINO,EMERGENCY,...,148.0,67.0,72.0,72.0,84.0,91.0,92.0,0.413793,0.379310,0.068966
2,14121,100058,297937,F,2139-09-08 21:38:00,2139-09-26 13:55:00,17.6785,56.8114,WHITE,EMERGENCY,...,108.0,59.0,58.0,57.0,79.0,78.0,77.0,0.636364,0.212121,0.000000
3,20683,100186,203786,M,2168-01-07 19:33:00,2168-01-12 14:00:00,4.7688,19.8138,WHITE,EMERGENCY,...,118.5,59.5,59.0,59.0,77.0,77.0,77.0,0.750000,0.583333,0.000000
4,81865,100215,248622,M,2193-05-03 20:47:00,2193-05-15 18:31:00,11.9056,48.0116,WHITE,EMERGENCY,...,113.0,69.0,66.0,66.0,82.0,80.0,81.0,0.818182,0.363636,0.090909


In [90]:
# df3.plot(x='mbp_24_inrange', y='hosp_mort')
# df3['hospital_mortality']= 1
# df3.loc[df3['hosp_mort']=='ALIVE', 'hospital_mortality']=0

In [91]:
# df3[['hosp_mort','hospital_mortality']].head(2)

,hosp_mort,hospital_mortality
0,EXPIRED,1
1,ALIVE,0


In [17]:
df3.to_csv("./csv_folder/mimic_table1_wth_range.csv")